In [2]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
import matplotlib.pyplot as plt
from loguru import logger
from matbench.bench import MatbenchBenchmark
from matbench.constants import CLF_KEY

from gptchem.gpt_regressor import GPTRegressor
from gptchem.tuner import Tuner

logger.enable("gptchem")

In [4]:
import time

import decorator


def retry(howmany, *exception_types, **kwargs):
    timeout = kwargs.get("timeout", 0.0)  # seconds

    @decorator.decorator
    def tryIt(func, *fargs, **fkwargs):
        for _ in range(howmany):
            try:
                return func(*fargs, **fkwargs)
            except exception_types or Exception as e:
                print(e)
                if timeout is not None:
                    time.sleep(timeout)

    return tryIt

In [5]:
mb = MatbenchBenchmark(
    autoload=True,
    subset=[
        "matbench_expt_gap",
        # "matbench_steels",
    ],
)

2023-02-03 08:45:19 INFO     Initialized benchmark 'matbench_v0.1' with 1 tasks: 
['matbench_expt_gap']


In [6]:
@retry(3, timeout=5)
def train_test_fold(task, fold):
    regressor = GPTRegressor(
        task.metadata["target"],
        Tuner(n_epochs=8, learning_rate_multiplier=0.02, wandb_sync=False),
        querier_settings={"max_tokens": 10},
    )
    train_inputs, train_outputs = task.get_train_and_val_data(fold)

    # train and validate your model
    regressor.fit(train_inputs, train_outputs.values)

    # Get testing data
    test_inputs = task.get_test_data(fold, include_target=False)

    # Predict on the testing data
    # Your output should be a pandas series, numpy array, or python iterable
    # where the array elements are floats or bools
    predictions = regressor.predict(test_inputs)

    # Record your data!
    task.record(fold, predictions)
    return predictions

In [7]:
from collections import defaultdict

In [11]:
predictions

defaultdict(list,
            {'matbench_expt_gap': [array([ 1.8 ,  0.  ,  0.  ,  0.21,  0.  , 10.4 ,  2.4 ,  2.6 ,  0.  ,
                      1.6 ,  2.  ,  1.7 ,  2.4 ,  0.  ,  0.  ,  0.  ,  0.  ,  2.  ,
                      0.  ,  0.  ,  0.  ,  0.  ,  3.76,  0.  ,  1.8 ,  0.  ,  0.  ,
                      2.27,  0.  ,  3.9 ,  5.17,  0.  ,  0.  ,  1.92,  1.2 ,  3.2 ,
                      2.37,  1.72,  1.35,  2.4 ,  2.3 ,  2.04,  3.17,  0.  ,  3.17,
                      1.72,  2.32,  4.17,  0.  ,  0.  ,  1.72,  0.  ,  0.  ,  0.  ,
                      1.3 ,  0.  ,  2.3 ,  2.3 ,  0.  ,  2.3 ,  0.  ,  1.58,  2.53,
                      1.88,  0.  ,  3.89,  3.3 ,  0.  ,  0.  ,  5.13,  0.  ,  1.76,
                      2.8 ,  2.3 ,  2.53,  2.17,  3.6 ,  0.  ,  3.2 ,  0.  ,  3.76,
                      0.  ,  0.  ,  0.  ,  3.3 ,  0.  ,  0.3 ,  0.  ,  2.6 ,  3.04,
                      2.3 ,  0.  ,  0.15,  0.  ,  0.  ,  2.  ,  0.  ,  0.  ,  2.3 ,
                      0.  ,  0.  ,  0

In [12]:
from fastcore.xtras import save_pickle

In [13]:
save_pickle("expt_gap_predictions.pkl", predictions)

In [8]:
predictions = defaultdict(list)

for task in mb.tasks:
    task.load()

    for fold_ind, fold in enumerate(task.folds):
        if task.is_recorded[fold_ind]:
            print(f"Skipping fold {fold_ind} of {task.dataset_name}")
            continue
        pred = train_test_fold(task, fold)
        predictions[task.dataset_name].append(pred)
        train_inputs, train_outputs = task.get_train_and_val_data(fold)

    # print(f"{task.dataset_name}: MAE  {task.scores['mae']['mean']}")

2023-02-03 08:45:23 INFO     Dataset matbench_expt_gap already loaded; not reloading dataset.


Upload progress: 100%|██████████| 421k/421k [00:00<00:00, 450Mit/s]


Uploaded file from /Users/kevinmaikjablonka/git/kjappelbaum/gptchem/experiments/04_regression/matbench/out/20230203_084523/train.jsonl: file-NWr8LeSxKwsDCTtHYgmFt1Ny


2023-02-03 08:45:27.296 | DEBUG    | gptchem.tuner:tune:188 - Requested fine tuning. {
  "created_at": 1675410327,
  "events": [
    {
      "created_at": 1675410327,
      "level": "info",
      "message": "Created fine-tune: ft-yIz4t695ZKn7tXVQL4Cso2bI",
      "object": "fine-tune-event"
    }
  ],
  "fine_tuned_model": null,
  "hyperparams": {
    "batch_size": null,
    "learning_rate_multiplier": 0.02,
    "n_epochs": 8,
    "prompt_loss_weight": 0.01
  },
  "id": "ft-yIz4t695ZKn7tXVQL4Cso2bI",
  "model": "ada",
  "object": "fine-tune",
  "organization_id": "org-TFRJXw3PPQocOWbu71eI2t9U",
  "result_files": [],
  "status": "pending",
  "training_files": [
    {
      "bytes": 420861,
      "created_at": 1675410327,
      "filename": "/Users/kevinmaikjablonka/git/kjappelbaum/gptchem/experiments/04_regression/matbench/out/20230203_084523/train.jsonl",
      "id": "file-NWr8LeSxKwsDCTtHYgmFt1Ny",
      "object": "file",
      "purpose": "fine-tune",
      "status": "uploaded",
      "

2023-02-03 09:37:41 INFO     Recorded fold matbench_expt_gap-0 successfully.


...

2023-02-03 15:34:10 INFO     Recorded fold matbench_expt_gap-3 successfully.


Upload progress: 100%|██████████| 421k/421k [00:00<00:00, 235Mit/s]


Uploaded file from /Users/kevinmaikjablonka/git/kjappelbaum/gptchem/experiments/04_regression/matbench/out/20230203_153410/train.jsonl: file-0DyHA941zMG5z60OCMTOtA3X


2023-02-03 15:34:14.019 | DEBUG    | gptchem.tuner:tune:188 - Requested fine tuning. {
  "created_at": 1675434853,
  "events": [
    {
      "created_at": 1675434853,
      "level": "info",
      "message": "Created fine-tune: ft-ubLeh9LTZYJSvLHFdlhEM6SN",
      "object": "fine-tune-event"
    }
  ],
  "fine_tuned_model": null,
  "hyperparams": {
    "batch_size": null,
    "learning_rate_multiplier": 0.02,
    "n_epochs": 8,
    "prompt_loss_weight": 0.01
  },
  "id": "ft-ubLeh9LTZYJSvLHFdlhEM6SN",
  "model": "ada",
  "object": "fine-tune",
  "organization_id": "org-TFRJXw3PPQocOWbu71eI2t9U",
  "result_files": [],
  "status": "pending",
  "training_files": [
    {
      "bytes": 420710,
      "created_at": 1675434853,
      "filename": "/Users/kevinmaikjablonka/git/kjappelbaum/gptchem/experiments/04_regression/matbench/out/20230203_153410/train.jsonl",
      "id": "file-0DyHA941zMG5z60OCMTOtA3X",
      "object": "file",
      "purpose": "fine-tune",
      "status": "uploaded",
      "

2023-02-03 16:27:44 INFO     Recorded fold matbench_expt_gap-4 successfully.


In [10]:
task.scores

ValueError: invalid __array_struct__

In [ ]:
mb.to_file("gpt_expt_gap_bench.json.gz")

2023-01-30 18:53:04 INFO     Successfully wrote MatbenchBenchmark to file 'gpt_steel_bench.json.gz'.
